In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import nltk
import sklearn.cluster
import distance

In [15]:
df = pd.read_csv('reformatted.csv', error_bad_lines=False)

In [19]:
coords = df.as_matrix(columns=['lat', 'lon'])

/Users/alexandertedeschi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [20]:
coords

array([[ 19.2170929, -98.9931659],
       [ 19.4339886, -99.0464413],
       [ 19.369749 , -99.14147  ],
       ...,
       [ 19.4265244, -99.1687519],
       [ 19.4071813, -99.1847452],
       [ 19.4246866, -99.1434637]])

In [72]:
kms_per_radian = 6371.0088
epsilon = .1 / kms_per_radian

In [73]:
db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

In [88]:
x=db.labels_
type(x)
df['cluster']=x

In [91]:
df.to_csv("clustered.csv")

Run `script.R` for preprocessing of the name column

In [11]:
df = pd.read_csv('preprocessed.csv')

words = df.loc[df['cluster'] == 50]['no_pharma'].tolist()
print(words)



['etc ', 'etc ', 'etc ', 'etc', 'etc ', 'etc ', 'etc ', ' similares', 'etc ', 'etc ', 'etc ', 'etc', 'etc ', '  similares']


In [12]:
words = np.asarray(words)
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])



In [17]:
affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity)

for cluster_id in np.unique(affprop.labels_):
    print(cluster_id)
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))




0
 - *etc :* etc 
1
 - *etc :* etc 
2
 - *etc:* etc
3
 - *etc :* etc 
4
 - * similares:*  similares
5
 - *etc:* etc
6
 - *  similares:*   similares


array(['  similares'], dtype='<U11')

In [75]:
cluster_labels = db.labels_

In [76]:
num_clusters = len(set(cluster_labels))

In [77]:
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])

In [78]:
print('Number of clusters: {}'.format(num_clusters))

Number of clusters: 155


In [79]:
clusters[1]

array([[ 19.4564308, -99.2153663],
       [ 19.4562374, -99.2155286],
       [ 19.4565604, -99.2155142],
       [ 19.455792 , -99.2157893],
       [ 19.4570467, -99.2150288],
       [ 19.4577189, -99.2146986]])